In [ ]:
import os
from sklearn.model_selection import train_test_split

img_dir = "rural/images_png"
mask_dir = "rural/masks_png"
train_txt = "rural/train.txt"
val_txt = "rural/val.txt"
split_ratio = 0.9

# 获取文件
img_files = sorted([f for f in os.listdir(img_dir) if f.endswith('.png')])
mask_files = sorted([f for f in os.listdir(mask_dir) if f.endswith('.png')])
assert len(img_files) == len(mask_files)

img_paths = [os.path.join("images_png", f) for f in img_files]
mask_paths = [os.path.join("masks_png", f) for f in mask_files]

# 划分
train_imgs, val_imgs, train_masks, val_masks = train_test_split(
    img_paths, mask_paths, train_size=split_ratio, random_state=42
)


# 写入
def write_list(txt_path, imgs, masks):
    with open(txt_path, "w") as f:
        for img, mask in zip(imgs, masks):
            f.write(f"{img} {mask}\n")


write_list(train_txt, train_imgs, train_masks)
write_list(val_txt, val_imgs, val_masks)

print("✅ 修复完成，已重写 train.txt 和 val.txt")


In [ ]:
!python train.py --config rural_seg.yml --save_dir output/rural_seg  --save_interval 500 --resume_model output\rural_seg\iter_3000

In [ ]:
!python train.py --config rural_seg_b4.yml --save_dir output/rural_seg_b4  --save_interval 1000

In [ ]:
!python train.py --config rural_seg_b2.yml --save_dir output/rural_seg_b2  --save_interval 1000

In [ ]:
!python train.py --config rural_seg_b2.yml --save_dir output/rural_seg_b2  --save_interval 1000

In [10]:
!python predict.py --config rural_seg.yml --model_path output/rural_seg_b2/iter_26000/model.pdparams --image_path "Rural/images_png/29.png" --save_dir output/rural_seg_b2

2025-04-04 17:46:56 [INFO]	
---------------Config Information---------------
batch_size: 2
iters: 40000
loss:
  coef:
  - 1
  types:
  - type: CrossEntropyLoss
lr_scheduler:
  end_lr: 1.0e-05
  learning_rate: 0.001
  power: 0.9
  type: PolynomialDecay
model:
  num_classes: 8
  pretrained: https://bj.bcebos.com/paddleseg/dygraph/mix_vision_transformer_b3.tar.gz
  type: SegFormer_B3
optimizer:
  momentum: 0.9
  type: sgd
  weight_decay: 4.0e-05
train_dataset:
  dataset_root: ./rural/
  mode: train
  num_classes: 8
  train_path: ./rural/train.txt
  transforms:
  - max_scale_factor: 2.0
    min_scale_factor: 0.5
    scale_step_size: 0.25
    type: ResizeStepScaling
  - crop_size:
    - 1024
    - 1024
    type: RandomPaddingCrop
  - type: RandomHorizontalFlip
  - brightness_range: 0.4
    contrast_range: 0.4
    saturation_range: 0.4
    type: RandomDistort
  - type: Normalize
  type: Dataset
val_dataset:
  dataset_root: ./rural/
  mode: val
  num_classes: 8
  transforms:
  - type: Normali

'C:\Program' 不是内部或外部命令，也不是可运行的程序
或批处理文件。
W0404 17:46:56.333611 20072 gpu_resources.cc:119] Please NOTE: device: 0, GPU Compute Capability: 8.9, Driver API Version: 12.6, Runtime API Version: 12.0
W0404 17:46:56.333611 20072 gpu_resources.cc:149] device: 0, cuDNN Version: 8.9.

1/1 [==============================] - 0s 403ms/step


In [ ]:
!python predict.py --config rural_seg.yml --model_path output/rural_seg/iter_38000/model.pdparams --image_path "../photo/12.png" --save_dir output/rural_seg 

In [ ]:
!python predict.py --config rural_seg.yml --model_path output/rural_seg/iter_40000/model.pdparams --image_path "D:\pythonProject\DeepSeek\ps3\PaddleSeg\Rural\images_png\901.png" --save_dir output/rural_seg 

In [ ]:
from skimage import io

io.imshow(r"D:\pythonProject\DeepSeek\ps3\PaddleSeg\Rural\masks_png\14.png")

In [ ]:
import os
import cv2
import numpy as np

mask_dir = 'rural/masks_png'
save_dir = 'rural/masks_png_fixed'

os.makedirs(save_dir, exist_ok=True)

for fname in os.listdir(mask_dir):
    if fname.endswith('.png'):
        path = os.path.join(mask_dir, fname)
        mask = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        fixed_mask = mask - 1  # 把 1-7 ➜ 0-6

        # 确保范围在合法值（防止负数或溢出）
        fixed_mask = np.clip(fixed_mask, 0, 6)

        cv2.imwrite(os.path.join(save_dir, fname), fixed_mask)


In [ ]:
import os
import paddle
from paddleseg.models.segformer import SegFormer
from paddleseg.models.backbones import MixVisionTransformer_B3
from paddleseg.core import train
from paddleseg.datasets import Dataset
from paddleseg.transforms import Compose

# === 配置参数 ===
NUM_CLASSES = 7  # 根据你的 mask 设置，0:背景, 1~4: 建筑/植被/水文/道路
CROP_SIZE = [1024, 1024]
BATCH_SIZE = 2
ITERS = 160000
LEARNING_RATE = 6e-5
SAVE_DIR = 'output/segformer_rural_b3'
PRETRAINED_URL = 'https://bj.bcebos.com/paddleseg/dygraph/backbone/mix_vision_transformer_b3.tar.gz'

# === Transforms ===
train_transforms = [
    {'type': 'Resize', 'target_size': CROP_SIZE},
    {'type': 'RandomHorizontalFlip'},
    {'type': 'Normalize'},
    {'type': 'ToTensor'}
]

val_transforms = [
    {'type': 'Resize', 'target_size': CROP_SIZE},
    {'type': 'Normalize'},
    {'type': 'ToTensor'}
]

# === Dataset 加载 ===
train_dataset = Dataset(
    transforms=train_transforms,
    dataset_root='rural',
    num_classes=NUM_CLASSES,
    mode='train',
    train_path='rural/train.txt',
    separator=' '
)

val_dataset = Dataset(
    transforms=val_transforms,
    dataset_root='rural',
    num_classes=NUM_CLASSES,
    mode='val',
    val_path='rural/val.txt',
    separator=' '
)

# === 构建模型 ===
model = SegFormer(
    backbone=MixVisionTransformer_B3(pretrained=PRETRAINED_URL),
    embedding_dim=768,
    num_classes=NUM_CLASSES
)

# === 优化器 ===
optimizer = paddle.optimizer.AdamW(
    learning_rate=LEARNING_RATE,
    parameters=model.parameters(),
    beta1=0.9,
    beta2=0.999,
    weight_decay=0.01
)

# === 启动训练 ===
train(
    model=model,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    optimizer=optimizer,
    save_dir=SAVE_DIR,
    iters=ITERS,
    batch_size=BATCH_SIZE,
    save_interval=1000,
    use_vdl=True,
    num_workers=4,
    losses={
        'types': [{'type': 'CrossEntropyLoss'}],
        'coef': [1.0]
    }
)
